# Env

In [ ]:
%%capture
!pip install thefuzz
!pip install llama-index
!pip install unidecode
!pip install llama-index-retrievers-bm25
!pip install llama-index-embeddings-huggingface
!pip install langchain
!pip install langchain-openai
!pip install gradio
!pip install langchain-community
!pip install deepeval

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''

# Load Data

In [ ]:
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_index.core import SimpleDirectoryReader

teacher_uet = pd.read_excel(hf_hub_download(repo_id="VictorJuiz/thesis",
                                      filename="Teacher_UET.xlsx",
                                      repo_type="dataset"))
teacher_uet.fillna('',inplace=True)
teacher_uet=teacher_uet.astype(str)

#************************************************************************************
paper_uet = pd.read_excel(hf_hub_download(repo_id="VictorJuiz/thesis",
                                      filename="Paper_UET.xlsx",
                                      repo_type="dataset"))

paper_uet.fillna('',inplace=True)
paper_uet=paper_uet.astype(str)

#************************************************************************************
thesis_uet = pd.read_excel(hf_hub_download(repo_id="VictorJuiz/thesis",
                                      filename="Thesis_UET.xlsx",
                                      repo_type="dataset"))
thesis_uet.fillna('',inplace=True)
thesis_uet=thesis_uet.astype(str)

#************************************************************************************
thesis_concate = pd.read_csv(hf_hub_download(repo_id="VictorJuiz/thesis",
                                      filename="Thesis_concate.csv",
                                      repo_type="dataset"))
thesis_concate.fillna('',inplace=True)
thesis_concate=thesis_concate.astype(str)

# ************************************************************************************
thesis_perform=hf_hub_download(repo_id="VictorJuiz/thesis",
                                      filename="Thesis_perform.txt",
                                      repo_type="dataset")

thesis_perform_data=SimpleDirectoryReader(input_files=[thesis_perform]).load_data()
print(thesis_perform_data)

[Document(id_='6d8494a6-9171-432e-89dd-d8bfb19dfb7b', embedding=None, metadata={'file_path': '/root/.cache/huggingface/hub/datasets--VictorJuiz--thesis/snapshots/5fd77801657a0150ebc0573dbfced4f363ed0adf/Thesis_perform.txt', 'file_name': 'Thesis_perform.txt', 'file_type': 'text/plain', 'file_size': 13620, 'creation_date': '2024-05-28', 'last_modified_date': '2024-05-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\ufeffI.    TIÊU CHUẨN ĐƯỢC LÀM KLTN\r\n−    Không nợ môn:                    Điểm trung bình ≥ 5.5\r\n−    Số đvht nợ từ 1 đến 4            Điểm trung bình ≥ 6.0\r\n−    Số đvht nợ từ 5 đến 8            Điểm trung bình ≥ 6.5\r\n−    Các sinh viên không đạt tiêu chuẩn trên nhưng nếu có nguyện vọng và có giảng viên trong Khoa nhận hướng dẫn 

In [ ]:
import random

test_thesis=thesis_uet[thesis_uet['Keyword']!='']
test_id=random.sample(test_thesis.index.to_list(),100)
test_thesis=test_thesis.loc[test_id]
thesis_uet=thesis_uet.drop(test_id)

In [ ]:
thesis_concate.head()

,Title,Link,Abstract,Keywords,Advisor
0,Đề xuất phương án nâng cấp hệ thống điều khiển...,https://dlib.hust.edu.vn/handle/HUST/14407,iới thiệu về mạng Profibus và cấu trúc mạng đi...,Hệ thống điều khiển; Giám sát,Hoàng Minh Sơn
1,Ứng dụng thuật toán logic mờ để điều khiển tốc...,https://dlib.hust.edu.vn/handle/HUST/14406,Khái quát về động lực học và các nhiễu ảnh hưở...,Logic mờ; Tàu thuỷ; Ứng dụng,Phạm Thượng Hàn
2,Nghiên cứu hiệu quả của công nghệ truyền tải đ...,https://dlib.hust.edu.vn/handle/HUST/14470,Tổng quan về công nghệ truyền tải điện một chi...,Hệ thống điện; Tải điện; Điện một chiều,Lã Văn Út
3,Phân tích ngắn mạch và nghiên cứu bảo vệ rơle ...,https://dlib.hust.edu.vn/handle/HUST/14471,Giới thiệu về nguồn điện phân tán. Tính toán n...,Bảo vệ rơ le; Ngắn mạch; Nguồn điện phân tán,Trần Bách
4,Sử dụng mô hình cháy AVL-MCC trên phần mềm mô ...,https://dlib.hust.edu.vn/handle/HUST/14468,Trình bày hệ thống luân hồi khí thải và lọc bụ...,Động cơ diesel,Lê Anh Tuấn


In [ ]:
personal_info=pd.DataFrame({
    'Name': ['Lê Thái Sơn'],
    'School': ['UET'],
    'Education program': 'Stardard',
    'Software Engineering': ['A'],
    'Discrete Mathematics': ['B+'],
    'Artificial Intelligent': ['A+'],
    'Algebra': ['B'],
    'Computer Network': ['C+'],
    'Total Credit': [114]
})
personal_info

,Name,School,Education program,Software Engineering,Discrete Mathematics,Artificial Intelligent,Algebra,Computer Network,Total Credit
0,Lê Thái Sơn,UET,Stardard,A,B+,A+,B,C+,114


# Fuzz

In [ ]:
import re
from unidecode import unidecode

def clean_text(text, mode=1):
    if mode==1:
        # keyword clean
        # clean keyword, keep -, white space, spilt by other punctuation
        result=re.split(r'[^\w\s-]',text)
    if mode==2:
        # Title clean
        # clean string, keep white space, spilt by other punctuation
        result=re.split(r'[^\w\s]',text)
    else:
        # clean name
        result=re.split(r'[^\w]',text)
    result=[' '.join(s.split()).lower() for s in result if s!='']
    if mode!=1:
        result=[unidecode(word) for word in result]
    return result

# clean_text("   web'\(trang web - 1090 !); very  \n good",mode=2)
# clean_text('Duc-Trong Le',mode=3)
clean_text('Kiến trúc nhiều tầng cho phát hiện và ngăn chặn trang web lừa đảo',mode=2)

['kien truc nhieu tang cho phat hien va ngan chan trang web lua dao']

In [ ]:
name_variation=[]
variation_origin={}
for idx,row in teacher_uet.iterrows():
    variation_list=row['Name variation'].split(',')
    for variation in variation_list:
        variation_origin[variation]=row['Tên']
    name_variation.extend(variation_list)

clean_title_set=[]
clean_raw_title={}
raw_title_set=paper_uet['Title'].to_list()+thesis_uet['Title'].to_list()+thesis_concate['Title'].to_list()
for raw_title in raw_title_set:
    clean_title=' '.join(clean_text(raw_title, mode=2))
    clean_raw_title[clean_title]=raw_title
    clean_title_set.append(clean_title)

In [ ]:
from thefuzz import fuzz,process

def search_name(name):
    name=' '.join(clean_text(name, mode=3))
    candidate=process.extractBests(name, name_variation,
                     limit=20,
                     scorer=fuzz.partial_token_sort_ratio)
    cnt={}
    for variation,score in candidate:
        key=variation_origin[variation]
        if key in cnt.keys():
            cnt[key]=cnt[key]+score
        else:
            cnt[key]=score
    # print(candidate)
    max_score=max(cnt.values())
    candidate=[key for key,value in cnt.items() if value==max_score]

    candidate=process.extractBests(name, candidate,
                     limit=20,
                     scorer=fuzz.token_set_ratio)
    final=[i[0] for i in candidate if i[1]==candidate[0][1]]
    return final,candidate[0][1]

# search_name('Hà Quang Thụy')
search_name('tran truong thuy')

(['Trần Trường Thủy'], 69)

In [ ]:
from thefuzz import fuzz,process

def search_title(title):
    title=' '.join(clean_text(title, mode=2))
    candidate=process.extractBests(title, clean_title_set,
                     limit=5,
                     scorer=fuzz.partial_ratio)
    final=[i[0] for i in candidate if i[1]==candidate[0][1]]
    final=[clean_raw_title[i] for i in final]
    return final,candidate[0][1]

search_title('Machine learning-based extraction of semantic relations from biomedical literature')

(['Machine learning-based extraction of semantic relations from biomedical literature = Trích xuất mối quan hệ ngữ nghĩa trong văn bản y sinh dựa trên học máy',
  'me+'],
 100)

# Chunking

In [ ]:
import pandas as pd
from llama_index.core import Document

paper_UET_data=[]
for idx,row in paper_uet.iterrows():
    metadata={'Authors': row['Authors']}
    paper_UET_data.append(Document(text='document: '+row['Title']+'. '+row['Description'], metadata=metadata))

#************************************************************************************
thesis_UET_data=[]
for idx,row in thesis_uet.iterrows():
    metadata={'Advisor': row['Advisor'],
              'Keywords': row['Keyword']}
    thesis_UET_data.append(Document(text='document: '+row['Title']+'. '+row['Abstract'], metadata=metadata))

#************************************************************************************
thesis_concate_data=[]
for idx,row in thesis_concate.iterrows():
    metadata={'Advisor': row['Advisor'],
              'Keywords': row['Keywords']}
    thesis_concate_data.append(Document(text='ducuent: '+row['Title']+ '. ' +row['Abstract'],  metadata=metadata))

# thesis_concate_data=thesis_concate_data[:3000]

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

splitter=TokenTextSplitter(chunk_size=1024*1024,chunk_overlap=0,separator=" ")

paper_uet_nodes=splitter.get_nodes_from_documents(paper_UET_data, show_progress=True)
thesis_uet_nodes=splitter.get_nodes_from_documents(thesis_UET_data, show_progress=True)
thesis_concate_nodes=splitter.get_nodes_from_documents(thesis_concate_data, show_progress=True)


splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
thesis_perform_nodes=splitter.get_nodes_from_documents(thesis_perform_data, show_progress=True)

Parsing nodes:   0%|          | 0/1560 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/2370 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/94940 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

# Indexing

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

# initialize storage context (by default it's in-memory)
# storage_context = StorageContext.from_defaults()
# storage_context.docstore.add_documents(nodes)

bm25_paper_uet = BM25Retriever.from_defaults(nodes=paper_uet_nodes,similarity_top_k=100)
bm25_thesis_uet = BM25Retriever.from_defaults(nodes=thesis_uet_nodes,similarity_top_k=100)
bm25_thesis_concate = BM25Retriever.from_defaults(nodes=thesis_concate_nodes,similarity_top_k=100)
bm25_thesis_perform = BM25Retriever.from_defaults(nodes=thesis_perform_nodes,similarity_top_k=100)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-small")

Settings.chunk_size = 256
Settings.chunk_overlap = 64
# Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = embed_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

vector_paper_uet = VectorStoreIndex(paper_uet_nodes, show_progress=True,
                                    storage_context=StorageContext.from_defaults())

vector_thesis_uet = VectorStoreIndex(thesis_uet_nodes, show_progress=True,
                                     storage_context=StorageContext.from_defaults())

vector_thesis_concate = VectorStoreIndex(thesis_concate_nodes, show_progress=True,
                                         storage_context=StorageContext.from_defaults())

vector_thesis_perform = VectorStoreIndex(thesis_perform_nodes, show_progress=True,
                                         storage_context=StorageContext.from_defaults())

# paper_uet_index.storage_context.persist()
# thesis_uet_index.storage_context.persist()
# thesis_concate_index.storage_context.persist()
# thesis_perform_index.storage_context.persist()

Generating embeddings:   0%|          | 0/1560 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/322 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/732 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7 [00:00<?, ?it/s]

# Create retriever

In [ ]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.postprocessor import LongContextReorder
from llama_index.core import QueryBundle


class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, vector_post_process,bm25_retriever, bm25_post_process,final_post_process):
        self.vector_retriever = vector_retriever
        self.vector_post_process = vector_post_process
        self.bm25_retriever = bm25_retriever
        self.bm25_post_process = bm25_post_process
        self.final_post_process = final_post_process
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_post_process.postprocess_nodes(self.bm25_retriever.retrieve(query, **kwargs))
        vector_nodes = self.vector_post_process.postprocess_nodes(self.vector_retriever.retrieve('query: '+query, **kwargs))

        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)

        for post_process in self.final_post_process:
            all_nodes = post_process.postprocess_nodes(all_nodes,query_bundle=QueryBundle('query: '+query))

        return all_nodes


final_post_process = [
    # CohereRerank(model="rerank-english-v2.0", api_key="Q7DQqKxPaBsdL44ZF77mO07SkUtcpmMRihrIq8RY",top_n=5),
    SentenceTransformerRerank(model="BAAI/bge-reranker-base",top_n=40),
    LongContextReorder()
]
hybrid_paper_uet = HybridRetriever(vector_paper_uet.as_retriever(similarity_top_k=100), SimilarityPostprocessor(similarity_cutoff=0.5), bm25_paper_uet,SimilarityPostprocessor(similarity_cutoff=0.1),final_post_process)
hybrid_thesis_uet = HybridRetriever(vector_thesis_uet.as_retriever(similarity_top_k=100), SimilarityPostprocessor(similarity_cutoff=0.5), bm25_thesis_uet, SimilarityPostprocessor(similarity_cutoff=0.1),final_post_process)
hybrid_thesis_concate = HybridRetriever(vector_thesis_concate.as_retriever(similarity_top_k=100), SimilarityPostprocessor(similarity_cutoff=0.5), bm25_thesis_concate, SimilarityPostprocessor(similarity_cutoff=0.1),final_post_process)
hybrid_thesis_perform = HybridRetriever(vector_thesis_perform.as_retriever(similarity_top_k=100), SimilarityPostprocessor(similarity_cutoff=0.1), bm25_thesis_perform, SimilarityPostprocessor(similarity_cutoff=0.1),final_post_process)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes =hybrid_thesis_concate._retrieve('web')
print(len(retrieved_nodes))
for node in retrieved_nodes:
    # display_source_node(node)
    print(node.metadata)

40
{'Advisor': '', 'Keywords': ''}
{'Advisor': '', 'Keywords': 'Internet users\nData mining\nWorld Wide Web, Use studies'}
{'Advisor': 'Nguyễn Bình Minh', 'Keywords': 'Web'}
{'Advisor': 'Cao Tuấn Dũng', 'Keywords': 'Web site'}
{'Advisor': '', 'Keywords': 'Web portals\nConsumers, Research\nConsumer behavior\nConsumer profiling\nOnline information services\nComputer literacy\nTelecommunication in education'}
{'Advisor': '', 'Keywords': ''}
{'Advisor': 'Phạm Huy Hoàng', 'Keywords': 'Website'}
{'Advisor': '', 'Keywords': ''}
{'Advisor': 'Cao Tuấn Dũng', 'Keywords': 'Web; Ứng dụng'}
{'Advisor': '', 'Keywords': ''}
{'Advisor': '', 'Keywords': ''}
{'Advisor': '', 'Keywords': ''}
{'Advisor': '', 'Keywords': ''}
{'Advisor': 'Nguyễn Thị Hoàng Lan', 'Keywords': 'Website; Công nghệ; Ứng dụng'}
{'Advisor': 'Cao Tuấn Dũng', 'Keywords': 'Web'}
{'Advisor': 'Brown, I\nMarres, N', 'Keywords': 'internet\nperformativity\nHIV\ncare\nhealth\nethics'}
{'Advisor': 'Huỳnh Quyết Thắng', 'Keywords': 'Phần mềm; M

In [ ]:
# from google.colab import files
# !zip -r /content/Chroma.zip /content/chromadb
# files.download("/content/Chroma.zip")

# Draft

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions

prompt=ChatPromptTemplate.from_messages([
    ('system', '{instruction}'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user','{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')

])
messages = prompt.format_messages(instruction='You are a helpful assistant',
                                  chat_history=[],
                                  input='Hi',
                                  agent_scratchpad=['i love you'])
print(messages)

[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='Hi'), HumanMessage(content='i love you')]


# Tools

In [ ]:
from langchain.agents import tool

@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    return query[::-1]

@tool
def get_personal_info():
    """Get personal learning information. Use when user asking information about themself"""
    context='Here is user information: \n'
    context=context+'Name: '+personal_info.loc[0,'Name']+'\n'+'Learning result:'+'\n'
    not_result=['Name','School']
    for col in personal_info.columns:
        if col in not_result:
            continue
        if col!='Total Credit':
            context=context+'- '+col+': '+str(personal_info.loc[0,col])+'\n'
        else:
            context=context+col+': '+str(personal_info.loc[0,col])+'\n'
    context=context+'\n\n'+'*'*20+'\n\n'
    return context

# print(get_personal_info.run({}))

In [ ]:
from pydantic.v1 import BaseModel,Field

class Intent(BaseModel):
    """Tag the piece of text with particular infor."""
    query: str = Field(..., description="information required by user")

@tool(args_schema=Intent)
def get_thesis_perform(query):
    """Get general information of doing thesis: process, qilification, etc"""
    retrieved=hybrid_thesis_perform._retrieve(query)
    context=''
    for node in retrieved:
        context=context+'\n'+node.text
    return context

# print(get_thesis_perform.run({'query': 'Ngày nộp khóa luận'}))

In [ ]:
class Object_Name(BaseModel):
    """Tag the piece of text with particular infor."""
    target: str = Field(..., description="Title of a work or name of a person")

@tool(args_schema=Object_Name)
def get_teacher_info(target):
    """Use it to get information about a specific teacher. Use it only when you are sure about the query intention."""

    name,score=search_name(target)
    context=''
    info=teacher_uet[teacher_uet['Tên'].isin(name)]
    interest=['Tên', 'Bộ môn', 'Chức danh', 'Hướng nghiên cứu', 'Chủ đề khóa luận', 'Liên hệ', 'HomePage', 'Link', 'Number of paper']
    for idx,row in info.iterrows():
        tmp=''
        for col in interest:
            if row[col]!='':
                tmp=tmp+col+": "+str(row[col])+'\n'
        context=context+tmp+'\n\n'

    return context

@tool(args_schema=Object_Name)
def get_single_work_info(target: str):
    """Use to get information about a specific paper or thesis given its title. Use it only when you are sure about the query intention."""
    clean_title=' '.join(clean_text(target,mode=2))
    print(clean_title)
    candidate=search_title(clean_title)
    clean_title=candidate[0][0]
    if candidate[1]<70:
        return 'There no work name '+ target +'\n'

    info=paper_uet[paper_uet['Title']==clean_title]
    if len(info)==0:
        info=thesis_uet[thesis_uet['Title']==clean_title]
    if len(info)==0:
        info=thesis_concate[thesis_concate['Title']==clean_title]
    if len(info)==0:
        return 'There no work name '+ target +'\n'

    interest=['Title','Abstract','Advisor','Link','Description','Authors']
    context=''
    for col in interest:
        if col in info.columns:
            context=context+col+': '+info.iloc[0][col]+'\n\n'

    context=context+'*'*20+'\n'
    return context

# print(get_teacher_info.run({'target': 'lsvinh'}))
# print(get_single_work_info.run({'target':'Machine learning-based extraction of semantic relations from biomedical literature'}))

In [ ]:
import random
import numpy as np

def sample_node(node_list,k=5, mode=1):
    if len(node_list)==0:
        return []
    if mode==1:
        score=[i.score for i in node_list]
    else:
        score=np.array([1.0 for i in node_list])

    num_samples = min(k, len(node_list))
    sampled_keys = np.random.choice(range(len(node_list)), size=num_samples, replace=False, p=score/sum(score))
    return [node_list[idx] for idx in sorted(sampled_keys)]

def retrieve_document(keyword,teacher, retriever):

    field=''
    separator=''

    if retriever==hybrid_paper_uet:
        field='Authors'
        separator=','
    else:
        field='Advisor'
        separator='\n'

    keyword=', '.join(clean_text(keyword))

    if keyword!='' and teacher!='':
        name,score=search_name(teacher)
        name_variation=[]
        for idx,row in teacher_uet[teacher_uet['Tên'].isin(name)].iterrows():
            name_variation.extend(row['Name variation'].split(','))

        retrieved=retriever._retrieve(keyword)
        document=[]

        for node in retrieved:
            value=node.metadata[field]
            if value=='':
                  continue
            name=[' '.join(clean_text(i,mode=1)) for i in value.split(separator)]
            name=[i for i in name if process.extractBests(i, name_variation, limit=1,scorer=fuzz.token_set_ratio)[0][1]>75]
            if len(name)!=0:
                document.append(node)

        return sample_node(document)
    elif keyword=='' and teacher!='':
        name,score=search_name(teacher)
        name_variation=[]
        for idx,row in teacher_uet[teacher_uet['Tên'].isin(name)].iterrows():
            name_variation.extend(row['Name variation'].split(','))
        # print(name_variation)
        doc_store=''
        if retriever==hybrid_paper_uet:
            doc_store=paper_uet_nodes
        elif retriever==hybrid_thesis_uet:
            doc_store=thesis_uet_nodes
        elif retriever==hybrid_thesis_concate:
            doc_store=thesis_concate_nodes
        document=[]
        for node in doc_store:
            for key, value in node.metadata.items():
                if key!=field:
                    continue
                if value=='':
                      continue
                name=[' '.join(clean_text(i,mode=1)) for i in value.split(separator)]
                exist_name=[]
                # print(name)
                for i in name:
                    score=[process.extractBests(i, name_variation, limit=1,scorer=fuzz.token_set_ratio)[0][1],
                         process.extractBests(i, name_variation, limit=1,scorer=fuzz.ratio)[0][1]]
                    for j in score:
                        if j>90:
                            exist_name.append(j)
                            break
                # name=[i for i in name if process.extractBests(i, name_variation, limit=1,scorer=fuzz.token_sort_ratio)[0][1]>85]
                # print(name)
                if len(exist_name)!=0:
                    document.append(node)
        return sample_node(document,mode=2)

    elif keyword!='' and teacher=='':
        retrieved=retriever._retrieve(keyword)
        return sample_node(retrieved)

    else:
        return []

# print(retrieve_document('','Lê Sỹ Vinh', hybrid_paper_uet))
# retrieve_document('machine leaning','thuy hg', hybrid_thesis_uet)
# retrieve_document('','tran trường thủy',hybrid_paper_uet)
# search_name('tran truong thuy')
# process.extractBests('thuytt', ['tran truong thuy', 'tt thuy', 'tt tran'], limit=1,scorer=fuzz.ratio)

In [ ]:
class Tagging(BaseModel):
    """tag the piece of text with particular infor."""
    keyword: str = Field(...,description="Phrases, that best expresses the document substance and type, separated by commas.")
    teacher: str = Field(...,description="Single name desired or suggested author, advisor of the query.")

@tool(args_schema=Tagging)
def contextualize_paper_uet_info(keyword: str,teacher: str) -> str:
    """Retrieve papers of uet"""
    document=retrieve_document(keyword,teacher,hybrid_paper_uet)
    # document=retrieve_document('','Lê Sỹ Vinh', hybrid_paper_uet)
    if len(document)==0:
        return ''
    context="Here is the list of paper that related to "+keyword
    if teacher!='':
        name,score=search_name(teacher)
        context=context+' and may contributed by '+', '.join(name)
    context=context+':\n\n'

    for node in document:
        tmp=node.text
        for key,value in node.metadata.items():
            tmp=tmp+'\n'+key+': '+value
        context=context+tmp+'\n\n'
    context=context+'*'*20
    return context

@tool(args_schema=Tagging)
def contextualize_thesis_uet_info(keyword: str,teacher: str) -> str:
    """Retrieve uet thesis"""
    document=retrieve_document(keyword,teacher,hybrid_thesis_uet)
    if len(document)==0:
        return ''
    context="Here is the list of UET thesis that related to "+keyword
    if teacher!='':
        name,score=search_name(teacher)
        context=context+' and may advised by '+', '.join(name)
    context=context+':\n\n'

    for node in document:
        tmp=node.text
        for key,value in node.metadata.items():
            tmp=tmp+'\n'+key+': '+value
        context=context+tmp+'\n\n'

    context=context+'*'*20
    return context

@tool(args_schema=Tagging)
def contextualize_thesis_concate_info(keyword: str,teacher: str) -> str:
    """Retrieve thesis that not belong to uet"""
    document=retrieve_document(keyword,teacher, hybrid_thesis_concate)
    if len(document)==0:
        return ''
    context="Here is the list of thesis that not belong to UET and may related to "+keyword
    context=context+':\n\n'

    for node in document:
        tmp=node.text
        for key,value in node.metadata.items():
            tmp=tmp+'\n'+key+': '+value
        context=context+tmp+'\n\n'
    context=context+'*'*20
    return context

@tool(args_schema=Tagging)
def contextualize_thesis_general(keyword: str,teacher: str) -> str:
    """Retrieve thesis when the query is not specific the thesis should belong to uet or not"""
    context_1=contextualize_thesis_uet_info.run({'keyword':keyword,'teacher':teacher})
    context_2=contextualize_thesis_concate_info.run({'keyword':keyword,'teacher':teacher})
    return context_1+context_2

# print(contextualize_paper_uet_info.run({'keyword': ' ', 'teacher': 'Lê Sỹ Vinh'}))
# print(contextualize_thesis_concate_info.run({'keyword' : '', 'teacher' : 'thuy qh'}))

In [ ]:
@tool(args_schema=Tagging)
def suggest_teacher(keyword: str,teacher: str):
    """Seaching for research direction of reacher. Use it only when user asking for suggest thesis advisor or ask for teacher with reseach related to some subject"""
    teacher_research=retrieve_document(keyword, '', hybrid_paper_uet)
    teacher_thesis=retrieve_document(keyword, '', hybrid_thesis_uet)
    candidate=[]

    for node in teacher_research:
        for key, value in node.metadata.items():
            if value=='':
                  continue
            name=[' '.join(clean_text(i,mode=1)) for i in value.split(',')]
            candidate.extend(name)
    for node in teacher_thesis:
        for key, value in node.metadata.items():
            if value=='':
                  continue
            name=[' '.join(clean_text(i,mode=1)) for i in value.split('\n')]
            candidate.extend(name)

    suggest_teacher=[]
    for i in candidate:
        tmp=process.extractBests(i, name_variation, limit=1,scorer=fuzz.token_set_ratio)
        if tmp[0][1]>80:
            suggest_teacher.append(tmp[0][0])

    suggest_teacher=[variation_origin[i] for i in suggest_teacher]
    suggest_teacher=list(set(suggest_teacher))

    context=f"""Here is the list of teachers that have research directions related to {keyword} : \n"""+', '.join(suggest_teacher)+'\n\n'

    return context

# print(suggest_teacher.run({'keyword':'web', 'teacher': ''}))

In [ ]:
@tool(args_schema=Tagging)
def suggest_thesis(keyword: str,teacher: str):
    """Suggest thesis topic. Only use when user asking for suggest a thesis topic"""
    context=''
    context=get_personal_info.run({})
    if teacher!='':
        context=context+contextualize_paper_uet_info.run({'keyword':keyword,'teacher':teacher})+'\n\n'

    context=context+contextualize_thesis_uet_info.run({'keyword':keyword,'teacher':teacher})+'\n\n'
    context=context+contextualize_thesis_concate_info.run({'keyword':keyword,'teacher': ''})+'\n\n'

    context=context+'You are given information about related uet thesis, others school thesis with related topic to know how a thesis look like. You may given advisor realted paper to know the research direction of the advisor. From user given description, advisor research direction and user learning result, sugggest a thesis\
 that fit with the user and use current modern technique. Write some effective, difficulty of that topic and,short instruction about how to work with that topic'
    return context

print(suggest_thesis.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))

Here is user information: 
Name: Lê Thái Sơn
Learning result:
- Education program: Stardard
- Software Engineering: A
- Discrete Mathematics: B+
- Artificial Intelligent: A+
- Algebra: B
- Computer Network: C+
Total Credit: 114


********************

Here is the list of paper that related to machine learning and may contributed by Hà Quang Thụy:

document: Learning to recognize phenotype candidates in the auto-immune literature using SVM re-ranking. The identification of phenotype descriptions in the scientific literature, case reports and patient records is a rewarding task for bio-medical text mining. Any progress will support knowledge discovery and linkage to other resources. However because of their wide variation a number of challenges still remain in terms of their identification and semantic normalisation before they can be fully exploited for research purposes.
This paper presents novel techniques for identifying potential complex phenotype mentions by exploiting a hybrid mod

In [ ]:
print(get_teacher_info.run({'target' : 'thuy qh'}))
print(get_single_work_info.run({'target':'Xây dựng công cụ định vị lỗi cho ứng dụng C/C++'}))

print(contextualize_paper_uet_info.run({'keyword' : 'machine learning','teacher' : ''}))
print(contextualize_paper_uet_info.run({'keyword' : '', 'teacher' : 'thuy qh'}))
print(contextualize_paper_uet_info.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))


print(contextualize_thesis_uet_info.run({'keyword' : 'machine learning','teacher' : ''}))
print(contextualize_thesis_uet_info.run({'keyword' : '', 'teacher' : 'thuy qh'}))
print(contextualize_thesis_uet_info.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))

print(contextualize_thesis_concate_info.run({'keyword' : 'machine learning','teacher' : ''}))
print(contextualize_thesis_concate_info.run({'keyword' : '', 'teacher' : 'thuy qh'}))
print(contextualize_thesis_concate_info.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))


print(contextualize_thesis_general.run({'keyword' : 'machine learning','teacher' : ''}))
print(contextualize_thesis_general.run({'keyword' : '', 'teacher' : 'thuy qh'}))
print(contextualize_thesis_general.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))


print(suggest_teacher.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))
print(suggest_thesis.run({'keyword': 'machine learning','teacher' : 'thuy qh'}))
print(get_thesis_perform.run({'query': 'Ngày nộp khóa luận'}))
print(get_personal_info.run({}))
print(create_your_own({'query': 'i love you'}))

Tên: Hà Quang Thụy
Bộ môn: Phòng thí nghiệm Tri thức
Chức danh: PGS
Hướng nghiên cứu: data mining and kdd, rough set mining data mining, text, web, etc, process mining lifelong learning business analysis
Liên hệ: thuyhq@vnu.edu.vn
HomePage: http://uet.vnu.edu.vn/~thuyhq/
Link: https://scholar.google.com/citations?hl=en&user=grsLl2wAAAAJ
Number of paper: 100.0



xay dung cong cu dinh vi loi cho ung dung c c
Title: Xây dựng công cụ định vị lỗi cho ứng dụng C/C++

Abstract: Kỹ thuật định vị lỗi dựa trên phổ (spectrum-based) là kỹ thuật được sử dụng và nghiên cứu phổ biến nhất. Tuy nhiên, các nghiên cứu không cung cấp công khai công cụ kèm theo hoặc các công cụ chưa hoàn chỉnh, khó tiếp cận và cải tiến cho việc định vị lỗi cho các chương trình C/C++. Mục tiêu của luận văn là nghiên cứu các kỹ thuật định vị lỗi hiện nay, từ đó đã lựa chọn ra kỹ thuật định vị lỗi dựa trên phổ phù hợp với các chương trình C/C++. Dựa trên kỹ thuật này, luận văn xây dựng công cụ hoàn chỉnh, có tên HiFa, định v

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


# Bind

In [ ]:
%%time
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models.openai import ChatOpenAI

tools=[
    get_teacher_info,
    get_single_work_info,
    contextualize_paper_uet_info,
    contextualize_thesis_uet_info,
    contextualize_thesis_concate_info,
    contextualize_thesis_general,
    suggest_teacher,
    suggest_thesis,
    get_personal_info,
    get_thesis_perform,
    # create_your_own
    ]

llm = ChatOpenAI(model="gpt-3.5-turbo",
                 temperature=0.0,
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()])

funtions=[convert_to_openai_function(i) for i in tools]
model_with_function=llm.bind(functions=funtions)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


CPU times: user 162 ms, sys: 4.01 ms, total: 166 ms
Wall time: 225 ms


In [ ]:
model_with_function.invoke('Tôi muốn tìm hiểu về bài báo Boo và tôi muốn có thông tin cá nhân của thầy lê sỹ minh')

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"target":"Boo"}', 'name': 'get_single_work_info'}}, response_metadata={'finish_reason': 'function_call'}, id='run-2f6306b7-dd74-4d60-afed-5ad006ade3ac-0')

In [ ]:
from langchain.schema.agent import AgentFinish
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

def route(result):
    if isinstance(result,AgentFinish):
        return result.return_values['output']
    else:
        tools={
            'get_teacher_info' : get_teacher_info,
            'get_single_work_info' : get_single_work_info,
            'contextualize_paper_uet_info' : contextualize_paper_uet_info,
            'contextualize_thesis_uet_info': contextualize_thesis_uet_info,
            'contextualize_thesis_concate_info' : contextualize_thesis_concate_info,
            'contextualize_thesis_general' : contextualize_thesis_general,
            'suggest_teacher' : suggest_teacher,
            'suggest_thesis' : suggest_thesis,
            'get_personal_info' : get_personal_info,
            'get_thesis_perform': get_thesis_perform
        }
        print(result.tool_input)
        return tools[result.tool].run(result.tool_input)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sasy assistance"),
    ("user", "{input}")
])

chain= prompt | model_with_function | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
print(chain.invoke('Tôi muốn tìm hiểu về bài báo Boo, tôi muốn có thông tin cá nhân của thầy lê sỹ minh'))
# print(chain.invoke('Hãy giới thiệu cho tôi một thầy cô có thể hướng dẫn khóa luận về web cho tôi'))

{'target': 'Boo'}
boo
Title: A new phylogenetic tree sampling method for maximum parsimony bootstrapping and proof-of-concept implementation

Link: https://scholar.google.com/citations?view_op=view_citation&hl=en&user=dZyMRT0AAAAJ&citation_for_view=dZyMRT0AAAAJ:Tyk-4Ss8FVUC

Description: Reconstructing maximum parsimony (MP) trees along with bootstrap for large multiple sequence alignments is a computationally challenging task. In this paper, we present MPBoot, a new method for approximating MP bootstrap based on the idea of a previously introduced method for maximum likelihood bootstrap. MPBoot economizes the computations by only sampling trees on the original alignment and simultaneously calculating exact parsimony scores for all bootstrap alignments. Preliminary experiments on simulated data showed that MPBoot is about 2-3.5 times faster and yields better parsimony scores than a fast bootstrap procedure implemented in TNT. MPBoot also produced less biased support value than TNT for 

# Chat agent (no streaming)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([
    ('system', '{instruction}'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user','{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')

])

In [ ]:
from langchain_openai import ChatOpenAI,OpenAI
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.callbacks.streaming_stdout_final_only import StreamingStdOutCallbackHandler

memory=ConversationBufferWindowMemory(k=10,return_messages=True,memory_key='chat_history',input_key='input', output_key="output")

def generate(message,chat_history,instruction,model,temperature,max_tokens):
    if len(chat_history)==0:
        memory.clear()

    llm = ChatOpenAI(model=model,
                     temperature=temperature,
                     max_tokens=max_tokens,
                     streaming=False,
                     callbacks=[StreamingStdOutCallbackHandler()])

    model_with_function=llm.bind(functions=funtions)
    agent_chain=RunnablePassthrough.assign(
        instruction = lambda x : instruction,
        agent_scratchpad= lambda x: format_to_openai_functions(x['intermediate_steps'])
    ) | prompt | model_with_function | OpenAIFunctionsAgentOutputParser()


    agent_executor=AgentExecutor(agent=agent_chain,tools=tools,memory=memory,
                                 chain_type='stuff',
                                 return_intermediate_steps=True,verbose=False,
                                 early_stopping_method='generate',
                                #  handle_parsing_errors=True, max_iterations=1
                                 )
    response=agent_executor.invoke({"input": message})
    chat_history = chat_history + [[message, response['output']]]
    context=[i[1] for i in response['intermediate_steps']]
    context=[[i,None] for i in context]
    return '', chat_history, context

In [ ]:
instruction="You are a  wonderful assistant who support student by give them the information. Your work is using provied information with your own knowledge to answer user question. You can access a various of tool to get infomation. Think carefully to choosse what tool do you need before use them. \
Make sure you understand the provided information. Do not say something that you not sure. Your answer must be gounded by the provieded infomration.  "

# Create generate test

In [ ]:
%%timeit
from tqdm.auto import tqdm

def create_dataset():
    dataset=pd.DataFrame({})
    dataset_idx=0
    for idx,row in tqdm(test_thesis.iterrows(),total=len(test_thesis)):
        advisors=row['Advisor'].split('\n')
        advisors.append('')
        for advisor in advisors:
            dataset.loc[dataset_idx,'Keywords']=row['Keyword']
            dataset.loc[dataset_idx,'Advisors']=advisor
            dataset.loc[dataset_idx,'expected']=row['Title']+row['Abstract']
            if advisor!='':
                dataset.loc[dataset_idx,'query']='Gợi ý cho tôi một đề tài khóa luận tốt nghiệp liên quan đến '+row['Keyword']+' được hướng dẫn bới thầy '+advisor
            else:
                dataset.loc[dataset_idx,'query']='Gợi ý cho tôi một đề tài khóa luận tốt nghiệp liên quan đến '+row['Keyword']
            _,chat_hsitory,context=generate(dataset.loc[dataset_idx,'query'],[],instruction,'gpt-3.5-turbo',0.5,1024)
            dataset.loc[dataset_idx,'context']='\n\n'.join(i[0] for i in context)
            dataset.loc[dataset_idx,'generate']=chat_hsitory[-1][1]
            dataset_idx=dataset_idx+1
    return dataset

gen_dataset=create_dataset()

  0%|          | 0/100 [00:00<?, ?it/s]

Based on the information provided, I would suggest the following thesis topic related to "Bộ điều khiển, Chíp, Công nghệ điện tử, Viễn thông, Động cơ" and supervised by Phạm, Thượng Cát:

Title: "Advanced Control Techniques for Permanent Magnet Synchronous Motor Drives in Electric Vehicles"

Description:
This thesis will focus on the application of advanced control techniques, specifically Model Predictive Control (MPC) and Sliding Mode Control (SMC), for the efficient operation of Permanent Magnet Synchronous Motors (PMSM) in electric vehicles. The research will involve the development and implementation of control algorithms to optimize the performance of PMSM drives in terms of efficiency, torque response, and speed control. The study will also explore the integration of power electronics components such as choppers and inverters for seamless operation of the motor drive system.

Research Difficulty:
- Implementing advanced control algorithms such as MPC and SMC for real-time applic

KeyboardInterrupt: 

In [ ]:
# gen_dataset=pd.read_excel('thesis_test.xlsx')

In [ ]:
import pandas as pd
from deepeval import evaluate
from deepeval import assert_test
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric,ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

answer_relevancy_metric = AnswerRelevancyMetric(model='gpt-3.5-turbo',
                                                threshold=0.07,
                                                async_mode=False,
                                                include_reason=True)
faithfulness_metric =  FaithfulnessMetric(threshold=0.7,
                                          model='gpt-3.5-turbo',
                                          async_mode=False,
                                          include_reason=True)
contextual_relevancy_metric = ContextualRelevancyMetric(threshold=0.7,
                                                        model="gpt-3.5-turbo",
                                                        include_reason=True)

test_set=[]
for idx, row in gen_dataset.iterrows():
    test_case = LLMTestCase(input=row['query'],
                            actual_output=row['generate'],
                            expected_output=row['expected']
                            retrieval_context=row['context'])
    test_set.append(test_case)

In [ ]:
evaluate([test_case], [answer_relevancy_metric,faithfulness_metric,contextual_relevancy_metric])
res=evaluate(test_set,
             [answer_relevancy_metric,faithfulness_metric,contextual_relevancy_metric],
             run_async=True,
             show_indicator=False)

df=pd.DataFrame()
for idx, metric in enumerate(res[0].metrics):
    df.loc[idx,'evaluation_cost']=metric.evaluation_cost
    df.loc[idx,'score']=metric.score
    df.loc[idx,'reason']=metric.reason

df.index=[i.__name__ for i in res[0].metrics]